# Empezamos la experimentación

In [1]:
from ultralytics import YOLO

**Entrenar YOLOv8 desde cero**

In [ ]:
# Cargar YOLO desde cero
model = YOLO('yolov8n.yaml')  

# Entrenar desde cero
model.train(data="/mnt/homeGPU/azapata/TFG/datasets/coco/coco.yaml", epochs=100, batch=16, pretrained=False, device="cuda")

**Evaluar el modelo**

In [ ]:
model.val(data="/mnt/homeGPU/azapata/TFG/datasets/coco/coco.yaml")